In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import json
import datetime

# 1) Read the GRIB file and calculate wind speeds from the u100 and v100 components
grib_file = "2015_test.grib"

# Open the GRIB file using xarray with cfgrib engine
ds = xr.open_dataset(grib_file, engine="cfgrib")

# Extract U and V wind components at 100m
u100 = ds["u100"].values  # U component (east-west)
v100 = ds["v100"].values  # V component (north-south)

# Compute wind speed from U and V components
wind_speeds = np.sqrt(u100**2 + v100**2)

C:\Users\alexa\AppData\Local\Temp\ipykernel_94564\1540551402.py:26: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq="H")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Get time information from GRIB file
forecast_gen_times = pd.to_datetime(ds.valid_time.values)  # Times when forecasts are valid

# 2) Create synthetic production data for 01.01.2014 to 31.12.2016
start_date = datetime.datetime(2014, 1, 1)
end_date = datetime.datetime(2016, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq="H")

# Generate synthetic production values between 0 and 1
production_values = np.random.rand(len(date_range))

# 3) Create a JSON file with 49 dictionaries for lead times (0, 3, 6, ..., 144 hours)
lead_times = np.arange(0, 145, 3)  # Lead times in hours

# list of dictionaries
lead_time_dicts = [{"lead_time": int(lt), "date_production_wind": []} for lt in lead_times]

for date, production in zip(date_range, production_values):
    for forecast_gen_times_block, wind_speed in zip(forecast_gen_times, wind_speeds):
        forecast_end_times_block = np.add(forecast_gen_times_block, pd.to_timedelta(lead_times, unit="h"))
        for i, forecast_end_time in enumerate(forecast_end_times_block):
            if forecast_end_time == date:
                lead_time = lead_times[i]
                lead_time_dicts["lead_time"==lead_time]["date_production_wind"].append([date, production, wind_speed])

C:\Users\alexa\AppData\Local\Temp\ipykernel_94564\124219706.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start=start_date, end=end_date, freq="H")


In [2]:
# Save the data to a JSON file
output_json_file = "forecast_production_data.json"
with open(output_json_file, "w", encoding="utf-8") as f:
    json.dump(lead_time_dicts, f, indent=4)

# Display the output JSON file content
import ace_tools as tools
tools.display_dataframe_to_user(name="Forecast and Production Data", dataframe=pd.DataFrame(lead_time_dicts))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()